# BNB_getBalances
As I do the majority of my trading on Binance a goal of mine has been to get to a point where I never have to manually log into Binance to check or change anything. This includes logging in to see my account balance. While all the live execution codes log their performance in text files and send email updates, checking these can be cumbersome. Not to mention as the number of strategies running in parallel increases, it is more important to focus on the overall return of the portfolio rather than the individual codes.
 
Thus I wrote a simple script to go through all Binance coins/tokens and return all non-zero values in both BTC and USD denominations. The account totals are also printed so that at any given time one can find the total portfolio value and individual holdings and allocations.

An example output is shown below:

In [1]:
from numpy import vstack
from pandas import DataFrame
from binance.client import Client
client = Client('API','KEY')

Balances = ['Asset', 'Amount', 'Locked', 'BTC Eqv.', '$ Eqv.']
info = client.get_account()
# print(info['balances'])
# print(len(info['balances']))
for i in range(0, len(info['balances'])):
    if(info['balances'][i]['free'] != '0.00000000' or info['balances'][i]['locked'] != '0.00000000'):
        Balances = vstack((Balances, [info['balances'][i]['asset'], float(info['balances'][i]['free']), float(info['balances'][i]['locked']), 0, 0]))

Balances = DataFrame(Balances[1:, :], columns=Balances[0, :])
# print(Balances)

for i in range(0, len(Balances)):
    if(Balances.loc[i, 'Asset'] == 'BTC'):
        Balances.loc[i, 'BTC Eqv.'] = float(Balances.loc[i, 'Amount']) + float(Balances.loc[i, 'Locked'])
    else:
        last = float(client.get_ticker(symbol=Balances.loc[i, 'Asset']+'BTC')['lastPrice'])
        Balances.loc[i, 'BTC Eqv.'] = (float(Balances.loc[i, 'Amount']) + float(Balances.loc[i, 'Locked']))*last

# print(Balances)
last = float(client.get_ticker(symbol='BTCPAX')['lastPrice'])

Balances['$ Eqv.'] = Balances['BTC Eqv.']*last
# print(last)
btc_balance = round(sum(Balances['BTC Eqv.']), 5)
usd_balance = round(sum(Balances['$ Eqv.']), 2)
print('All Balances\n{}\n'.format(Balances))
print('Total BTC: {}\nTotal $: {}\n'.format(btc_balance, usd_balance))


All Balances
  Asset      Amount   Locked     BTC Eqv.    $ Eqv.
0   BTC  0.07105488  0.00175    0.0728049   276.864
1   LTC     0.00457      0.0  5.71296e-05  0.217253
2   BNB  0.54638334      0.0   0.00164959   6.27308
3   ETC       13.97      0.0    0.0153949   58.5442
4  DASH    1.093995      0.0      0.02351   89.4041
5   XMR    1.243971      0.0     0.015638   59.4683
6   XRP       325.0      0.0    0.0266077   101.184

Total BTC: 0.15566
Total $: 591.96

